# RandomForest Classifier Implementation with PipeLines and Hyperparameter Tuning

#### Importing Dataset

In [45]:
import seaborn as sns
df = sns.load_dataset('tips')

In [46]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [47]:
#Target Column
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [48]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

#### Independent And Dependent Feature

In [49]:
X = df.drop('time',axis=1)
y = df['time']

In [50]:
X

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4
...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,3
240,27.18,2.00,Female,Yes,Sat,2
241,22.67,2.00,Male,Yes,Sat,2
242,17.82,1.75,Male,No,Sat,2


#### Train Test Split

In [51]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=42)

In [52]:
X_train

,total_bill,tip,sex,smoker,day,size
228,13.28,2.72,Male,No,Sat,2
208,24.27,2.03,Male,Yes,Sat,2
96,27.28,4.00,Male,Yes,Fri,2
167,31.71,4.50,Male,No,Sun,4
84,15.98,2.03,Male,No,Thur,2
...,...,...,...,...,...,...
106,20.49,4.06,Male,Yes,Sat,2
14,14.83,3.02,Female,No,Sun,2
92,5.75,1.00,Female,Yes,Fri,2
179,34.63,3.55,Male,Yes,Sun,2


In [53]:
from sklearn.impute import SimpleImputer #Handling Missing Values
from sklearn.preprocessing import OneHotEncoder #Handling Categorical Features
from sklearn.preprocessing import StandardScaler #Feature Scaling
from sklearn.pipeline import Pipeline #Pipeline
from sklearn.compose import ColumnTransformer #Wrapper for the all the pipelines:)

In [54]:
#Splitting categorical and numerical features
categorical_features = ['sex','smoker','day']
numerical_features = ['total_bill','tip','size']

#### PipeLine (Feature Engineering)

In [55]:
numerical_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='median')), #missing values
        ('scaler',StandardScaler()) #feature scaling
    ]
)

categorical_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='most_frequent')), #handling missing values
        ('onehotencoder',OneHotEncoder()) #categorical features to numerical
    ]
)

In [37]:
#wrapper for all the pipelines (columntranfer is used)

preprocessor = ColumnTransformer(
    [
        ('numerical_pipeline',numerical_pipeline,numerical_features),
        ('categorical_pipeline',categorical_pipeline,categorical_features)
    ]
)

In [56]:
X_train = preprocessor.fit_transform(X_train)


In [57]:
X_test = preprocessor.transform(X_test)

#### Model Training

In [58]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [60]:
#Automate Model Training Process
models = {
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'SVC':SVC()
}

In [61]:
from sklearn.metrics import accuracy_score

In [68]:
def evaluate_model(X_train,X_test,y_train,y_test,models):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        #Train the model
        model.fit(X_train,y_train)
        #Predicting the test data
        y_test_pred = model.predict(X_test)
        #Accuracy of test data
        test_model_accuracy = accuracy_score(y_test,y_test_pred)
        #Adding to accuracy and corresponding model name to the dictionary Report
        report[list(models.keys())[i]]=test_model_accuracy
        
    return report
    
    
    
    
    
    

In [69]:
#Accuracy of each model
evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random Forest': 0.9591836734693877,
 'Decision Tree': 0.9387755102040817,
 'SVC': 0.9591836734693877}

#### Insights
Choosing best model as Random Forest

In [70]:
classifier = RandomForestClassifier()

#### Hyperparameter Tuning

In [71]:
params={
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
    }

In [72]:
from sklearn.model_selection import RandomizedSearchCV

In [73]:
cv = RandomizedSearchCV(classifier,param_distributions=params,cv=5,scoring='accuracy',verbose=3)

In [74]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=200;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=200;, score=1.000 total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=200;, score=0.923 total time=   0.1s
[CV 1/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.974 total time=   0.1s
[CV 2/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.923 total time=   0.1s
[CV 3/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.974 total time=   0.1s
[CV 4/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.949 total time=   0.1s
[CV 5/5] END criterion=entropy, max_depth=5, n_estimators=200;, score=0.949 total time=   0.1s

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [75]:
cv.best_params_

{'n_estimators': 200, 'max_depth': 3, 'criterion': 'entropy'}

In [79]:
clf = RandomForestClassifier(n_estimators=200,max_depth=3,criterion='entropy')

In [80]:
clf.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=3, n_estimators=200)

In [81]:
y_test_pred = clf.predict(X_test)

In [82]:
accuracy_score(y_test,y_test_pred)

1.0